https://adventofcode.com/2022/day/23

In [11]:
import numpy as np
from dataclasses import dataclass

In [9]:
def load_data(filename):
    with open(filename, 'r') as f:
        lines = f.read().split('\n')
    return [list(line) for line in lines]

In [13]:
filename = "../example_data/day23_example_data.txt"
data = load_data(filename)

@dataclass
class Elf():
    loc: np.ndarray

elves = []
for y, row in enumerate(data):
    for x, s in enumerate(row):
        if s == '#':
            elves.append(Elf(loc=np.array([x, y])))
            



In [14]:
elves

[Elf(loc=array([4, 0])),
 Elf(loc=array([2, 1])),
 Elf(loc=array([3, 1])),
 Elf(loc=array([4, 1])),
 Elf(loc=array([6, 1])),
 Elf(loc=array([0, 2])),
 Elf(loc=array([4, 2])),
 Elf(loc=array([6, 2])),
 Elf(loc=array([1, 3])),
 Elf(loc=array([5, 3])),
 Elf(loc=array([6, 3])),
 Elf(loc=array([0, 4])),
 Elf(loc=array([2, 4])),
 Elf(loc=array([3, 4])),
 Elf(loc=array([4, 4])),
 Elf(loc=array([0, 5])),
 Elf(loc=array([1, 5])),
 Elf(loc=array([3, 5])),
 Elf(loc=array([5, 5])),
 Elf(loc=array([6, 5])),
 Elf(loc=array([1, 6])),
 Elf(loc=array([4, 6]))]